# TMA4215 - Assignment 2 - Conjugate Gradient and ODEs - Solution

In this assignment you will implement the Preconditioned Conjugate Gradient (PCG) method.
Then there are some small theoretical questions on Numerical ODEs.


**Problem 1.** 
**(a)** Fill in the templates below for a function *PCG* and a function *PrecJac* to be passed as an argument to *PCG*. 
Also allow for *no preconditioning* e.g. by implementing a function *PrecNull* that just returns it input argument.

*PCG* should take 5 input arguments, 
1. the function *A_func* that takes $v$ as input and returns $A\cdot v$.
2. the right hand side *b* of the system $A\cdot x = b$ (numpy array)
3. The initial guess *x0*
4. *tol*, a tolerance used to define the stopping criterion
5. *Precond* which is a function that takes a vector $v$ as input and returns $P^{-1}v$ for the preconditioner $P$.

There should be 3 output arguments: 
1. the computed solution $x$, 
2. the residual after the last iteration, $r$ 
3. the number of iterations *iter* needed to meet the stopping criterion 

The stopping criterion can be taken as ($\|r\|<$*tol* or *iter*$\geq N$) where $N$ is the dimension of the problem.

**Solution:** See the code cell below




In [3]:
import numpy as np
import numpy.linalg as la

# These are the functions to be implemented in 1a
def PCG(A_func,b,x0,tol,Precond):
    N=len(x0)
    x=x0
    r = b - A_func(x)
    z = Precond(r)
    p = z
    Ax = A_func(x)
    zr = np.inner(z,r)
    rnrm = la.norm(r,2)
    iter=0
    while rnrm > tol and iter < N:
        iter+=1
        Ap = A_func(p)
        alpha = zr/np.inner(p,Ap)
        x = x + alpha*p
        r = r - alpha*Ap
        z = Precond(r)
        zr1 = np.inner(z,r)
        beta = zr1/zr
        p=z+beta*p
        zr=zr1
        rnrm = la.norm(r,2)
        
    return x, iter, r
    
    
def PrecNull(r):
    return r

def PrecJac(p):
    n=len(p)
    u=np.zeros((n,))
    for k in range(nJac):
        r=p-A_func(u)
        u=u+omega*0.25*r
    return u

**(b)** Write a Gauss-Jacobi preconditioner function (see below *PrecJac*)
Test your implementation using the supplied function *A_func* which takes as input a vector $v\in\mathbb{R}^{n^2}$ and returns the vector in $A\cdot v\in\mathbb{R}^{n^2}$. This corresponds to a discretisation of the Poisson equation

$$ -\nabla^2 u = f $$

in the plane, so that there are $n$ discretisation points in each direction, thereby dimension $n^2$ of the unknown solution vector. You can take *x0* to be the 0-vector in $\mathbb{R}^{n^2}$. The right hand side is also obtained by a supplied function *b_def* (see below) that from an input integer $n$ returns a vector $b\in\mathbb{R}^{n^2}$.

*PrecJac(v)*. This function can be considered as tailored for the example given by *A_func*.
Starting with the 0-vector as initial guess, apply a specified number of iterations with the Gauss-Jacobi iterative method to the problem $Au=v$. The number of iterations can be specified in the main program. 
You need the diagonal of $A$ to construct the preconditioner and you can assume that $M=\text{diag}(A)=4I$ (four times the identity matrix). Therefore, via $A=M-N$ in Gauss-Jacobi, you can write $N=M-A=4I-A$ so that the GJ-iteration becomes

$$
     u^{(k+1)} = M^{-1}N u^{k} + M^{-1}b = (I-\frac14 A)u^{(k)}+\frac 14 b = u^{(k)}+\frac14 (b-Au^{(k)}),\quad
     u^{(0)} = 0
$$

Try to run the program with and without preconditioner for $N=n^2, \ n=20, 40, 60, \ldots, 200$. For each $n$, print out a line with $n$, *iterNull* and *iterJac2*, and *iterJac4*. Here *iterNull* is without preconditoning,
*iterJac2* and *iterJac4* is with 2 and 4 Gauss-Jacobi iterations respectively.

You can use *tol*$=10^{-5}$ in all experiments. Always let *x0* be the zero-vector.

**Control question 1.** How many CG iterations are needed with no preconditioning, $N=n^2=256 (n=16)$ with *tol* and *x0* as specified above.

**Control question 2.** How many PCG iterations are needed with preconditioning, 4 Gauss-Jacobi iterations, $N=n^2=256 (n=16)$ with *tol* and *x0* as specified above.

**Solution:** See the code cell above for the preconditioner. See the code under the functions A_func and b_def for the rest.

In [17]:
import numpy as np
import sys


def A_func(v):
    n2=len(v)
    n=int(np.sqrt(n2))
    if n**2 != n2:
        sys.exit("Size of vector must be a square")
    M=np.zeros((n+2,n+2))
    M[1:n+1,1:n+1]=np.reshape(v,(n,n))
    AM = 4*M[1:n+1,1:n+1]-M[:n,1:n+1]-M[2:n+2,1:n+1]-M[1:n+1,:n]-M[1:n+1,2:n+2]
    Av = np.reshape(AM,(n2,))
    return Av

def b_def(n):
    h = 1/(n+1)
    X = np.linspace(h,1-h,n)
    Y=X
    B = np.outer( 16*X**2*(1-X)**2, 16*Y**2*(1-Y)**2 )
    b = np.reshape(B, (n**2,))
    return B,b

# Here we answer the questions of 1b and control questions
# First do the loop over all the n-values
tol=1e-5
print('{0:>6s} {1:>6s} {2:>6s} {3:>6s}'.format('n','iter','iterJ2','iterJ4'))
print('-----------------------------')
for k in range(10):
    n=20*(k+1)
    B,b = b_def(n)
    x0=np.zeros((n**2,))
    x, iter, r = PCG(A_func, b, x0, tol, PrecNull)
    nJac=2
    xJ2, iterJ2, rJ2 = PCG(A_func, b, x0, tol, PrecJac)
    nJac=4
    xJ4, iterJ4, rJ4 = PCG(A_func, b, x0, tol, PrecJac)
    print('{0:6d} {1:6d} {2:6d} {3:6d}'.format(n, iter, iterJ2, iterJ4))




# First control question
print('\n\nControl question 1')
n=16
B,b=b_def(n)
x0=np.zeros((n**2,))
tol=1e-5
x, iter, r = PCG(A_func, b, x0, tol, PrecNull)
print('Answer:',iter)

# Second control question
print('\n\nControl question 2')
n=16
nJac=4
B,b=b_def(n)
x0=np.zeros((n**2,))
tol=1e-5
xJ4, iterJ4, rJ4 = PCG(A_func, b, x0, tol, PrecJac)
print('Answer:',iterJ4)









     n   iter iterJ2 iterJ4
-----------------------------
    20     26     14     11
    40     53     28     20
    60     81     42     30
    80    109     56     40
   100    137     70     50
   120    168     84     60
   140    198    100     70
   160    226    115     81
   180    255    129     92
   200    283    143    102


Control question 1
Answer: 21


Control question 2
Answer: 9


**Problem 2.** 

**(a)** Find the stability function of the Runge-Kutta method whose Butcher tableaux is given as

$$
\begin{array}{c|rr}
\frac12 & \frac12 & 0 \\
\frac32 & -\frac12 & 2 \\ \hline
        & -\frac12 & \frac32
\end{array}
$$

**Solution:** This can be computed in (at least) two ways. One can work out what is $K_1$, $K_2$ and then $y_{n+1}$ when the method is applied to the test equation $y'=\lambda y$. Alternatively one can evaluate the expression
$$
   R(z) = 1 + zb^T(I-zA)^{-1}\mathbf{1}
$$

In either case one should get

$$
R(z)=\frac{z-1}{2z-1}
$$






**(b)** Does the stability region of this method contain the left half plane, $\mathbb{C}^-=\{z\in\mathbb{C}: \text{Re}(z)<0\}$. 

**Solution:** By requiring $|R(\alpha+i\beta)|^2 \leq 1$ (where $\alpha, \beta \in\mathbb{R}$) we get after some calculations

$$
   3\alpha^2 + 3\beta^2 - 2\alpha \geq 0\quad\Rightarrow\quad \left(\alpha-\frac13\right)^2+\beta^2 \geq \left(\frac13\right)^2
$$

which corresponds the stability region $S_R=\{z\in\mathbb{C}: |z-\frac13|\geq\frac13\}$
This is everything outside a disc centered in $z=\frac13$ with radius $\frac13$, and this region contains the negative half plane $\mathbb{C}^-$.


